In [23]:
from IPython.utils import io
import csv
with io.capture_output() as captured:  
   %run ../../2-FeatureExtraction/ContextualEmbeddings/ContextualEmbeddings.ipynb import train_contextual_embeddings, valid_contextual_embeddings, train_labels_2, valid_labels_2, train_labels_1, valid_labels_1

In [14]:
from torch import nn
from torch.optim import Adam
import torch
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import re
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import numpy as np

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu"); print(device)

cpu


#### Loading the Data

In [16]:
train_data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y2))
valid_data = TensorDataset(torch.from_numpy(X), torch.from_numpy(Y2))

batch_size = 128

train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(valid_data, shuffle=True, batch_size=1024)

#### Implementing the Model Head

In [17]:
class ContextualClassifier(nn.Module):

  def __init__(self, layer_1=768, layer_2=512, output=10):
    super(ContextualClassifier, self).__init__()
    self.linear_1 = nn.Linear(layer_1, layer_2)
    self.linear_2 = nn.Linear(layer_2, output)
    self.relu = nn.ReLU()

  def forward(self, contextualized_embeddings):
    linear_1_output = self.linear_1(contextualized_embeddings)
    linear_2_output = self.linear_2(self.relu(linear_1_output))
    output_layer = self.relu(linear_2_output)
    return output_layer


#### Initializing the Model

In [18]:
EPOCHS = 300
LR = 3e-6
N1 = 768
N2 = 512
WeightedLoss = False
w = torch.tensor(np.load("../../Dataset/w1.npy").astype(np.float32)).to(device)
w = torch.tensor([0.3, 0.25, 0.7])

ModelInfo = {
   "Model": "Arabert-2",
   "batch_size": batch_size,
   "Hidden Dimension 1":N1,
   "Hidden Dimension 2": N2,
   "Learning Rate": LR,
   "Number of Epochs": EPOCHS,
   "Weighted Loss": WeightedLoss,
   "Weights": w
}

model = ContextualClassifier(layer_1=N1, layer_2=N2).to(device)

In [19]:
if WeightedLoss:
    criterion = nn.CrossEntropyLoss(weight=w).to(device)
else:
    criterion = nn.CrossEntropyLoss().to(device)

optimizer = Adam(model.parameters(), lr=LR)

#### Training the Model

In [20]:
for epoch_num in tqdm(range(EPOCHS)):
  for train_input, train_label in train_loader:
    model.zero_grad()

    train_input = train_input.to(device)
    train_label = train_label.to(device)
    output = model(train_input)
    output = output.squeeze(1)
    batch_loss = criterion(output, train_label)

    batch_loss.backward()
    optimizer.step()

100%|██████████| 300/300 [04:23<00:00,  1.14it/s]


#### Validating the Model

In [24]:
output_labels = []

with torch.no_grad():
  for valid_input, valid_label in tqdm(valid_loader):
    valid_input = valid_input.to(device)
    valid_label = valid_label.to(device)
    output = model(valid_input).cpu()
    output = output.squeeze(1)
    output_labels.extend(output.argmax(dim=1).tolist())

print()
F1 = f1_score(Y2, output_labels, average='macro')
Report = classification_report(Y2, output_labels, zero_division=0, output_dict=True)
print(classification_report(Y1, output_labels, zero_division=0))

100%|██████████| 1/1 [00:00<00:00, 16.13it/s]


              precision    recall  f1-score   support

           0       0.00      0.00      0.00       126
           1       0.81      0.28      0.42       804
           2       0.10      0.17      0.13        70
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.24      1000
   macro avg       0.11      0.06      0.07      1000
weighted avg       0.66      0.24      0.35      1000



#### Saving the Model

In [25]:
with open('runs.csv', 'a') as f:  
      run_info = {**Preprocessing, **Features, **ModelInfo, "acc":Report["accuracy"],"BF1": Report["macro avg"]["f1-score"], "WF1": Report["weighted avg"]["f1-score"] }
      w = csv.DictWriter(f, run_info.keys())
      w.writeheader()
      w.writerow(run_info)